In [1]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd

In [30]:
class RbcParser:
    def __init__(self):
        pass

    def get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей

        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        soup = bs(rq.get("https://www.rbc.ru/v10/ajax/get-news-by-filters/", param_dict).json()["html"], "lxml")
        return pd.DataFrame([{"title": ' '.join(soupchik.find("span", {"class": "item__title rm-cm-item-text"}).text.split()),
                              "href": soupchik["href"],
                              "text": self.get_article_data(soupchik["href"])}
                             for soupchik in soup.find_all("a", {"class": "item__link"})])


    def get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml")


        text = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))

        return text

    def get_articles(self, param_dict) -> pd.DataFrame:
        """
        param_dict: dict
        ### Параметры запроса
        ###### category - категория поиска
        ###### offset - смещение поисковой выдачи
        ###### step - шаг подгрузки, 12
        ###### limit - лимит статей
        """

        offset = param_dict["offset"]
        limit = param_dict["limit"]
        step = param_dict["step"]
        category = param_dict["category"]
        request_params = {
            "limit": step,
            "category": category
            }

        out = pd.DataFrame()

        while offset < limit:
            request_params["offset"] = offset


            print(f'Parsing articles from {offset} to {offset+step}')
            out = pd.concat([out, self.get_search_table(request_params)], ignore_index=True)
            print(f"Total news: {len(out)}")
            offset += step

        print('Finish')

        return out

In [43]:
# Задаем параметры запросы и складываем в param_dict
param_dict = \
    {
        'category': 'business',
        'offset': 0,
        'limit': 30,
        'step': 12
    }


print("param_dict:", param_dict)

param_dict: {'category': 'business', 'offset': 0, 'limit': 30, 'step': 12}


In [44]:
# Действует ограничение в 10 статей на 1 запрос (параметром limit)

parser = RbcParser()
# Пример работы программы итеративного сбора большого количества текстов статей
# Работает, конечно, очень долго :(
table = parser.get_articles(param_dict = param_dict)
table.to_csv("rbk_last_news.csv")
table.head()

Parsing articles from 0 to 12
Total news: 12
Parsing articles from 12 to 24
Total news: 24
Parsing articles from 24 to 36
Total news: 36
Finish


,title,href,text
0,Туроператоры оценили возможное количество тури...,https://www.rbc.ru/business/08/10/2022/6341392...,В Крыму сейчас может находиться порядка 50 тыс...
1,Компания Абрамовича купила здание столовой дом...,https://www.rbc.ru/business/08/10/2022/633ec7b...,"Компания «Парк девелопмент», подконтрольная Mi..."
2,ЕС запретил ввоз пластмассы из России почти на...,https://www.rbc.ru/business/08/10/2022/633ff8b...,Евросоюз в рамках восьмого пакета санкций прот...
3,Путин отстранил «дочку» ExxonMobil от проекта ...,https://www.rbc.ru/business/07/10/2022/634071e...,Президент Владимир Путин подписал указ о созда...
4,«Кузбассразрезуголь» рассказал о проблемах с п...,https://www.rbc.ru/business/07/10/2022/634021d...,Второй крупнейший производитель угля в России ...


In [46]:
pd.read_csv("rbk_last_news.csv", index_col=0)

,title,href,text
0,Туроператоры оценили возможное количество тури...,https://www.rbc.ru/business/08/10/2022/6341392...,В Крыму сейчас может находиться порядка 50 тыс...
1,Компания Абрамовича купила здание столовой дом...,https://www.rbc.ru/business/08/10/2022/633ec7b...,"Компания «Парк девелопмент», подконтрольная Mi..."
2,ЕС запретил ввоз пластмассы из России почти на...,https://www.rbc.ru/business/08/10/2022/633ff8b...,Евросоюз в рамках восьмого пакета санкций прот...
3,Путин отстранил «дочку» ExxonMobil от проекта ...,https://www.rbc.ru/business/07/10/2022/634071e...,Президент Владимир Путин подписал указ о созда...
4,«Кузбассразрезуголь» рассказал о проблемах с п...,https://www.rbc.ru/business/07/10/2022/634021d...,Второй крупнейший производитель угля в России ...
5,Производитель бумаги SvetoCopy продал бизнес в...,https://www.rbc.ru/business/07/10/2022/63400af...,"Американская компания Sylvamo, которая выпуска..."
6,«Северсталь» опровергла передачу доли Мордашов...,https://www.rbc.ru/business/07/10/2022/634010b...,В «Северстали» опровергли сообщение о планах А...
7,Новым владельцем Lamoda станет собственник уни...,https://www.rbc.ru/business/07/10/2022/633ffed...,Владелец онлайн-магазина Lamoda — сингапурская...
8,Власти обсудят новую программу по замене элект...,https://www.rbc.ru/business/07/10/2022/633ee81...,Для развития городского наземного электрическо...
9,Переориентация на внутренний рынок: новые реал...,https://www.rbc.ru/business/07/10/2022/6336efb...,"По данным Минпромторга России, производство ме..."
